# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***Long Short-Term Memory (LSTM)*** 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


### Training and Evaluation process

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.utils.data. <br>

In this case, the data was treated as *timeseries* with sequence length equal to 2. In order for that to be able to happen, some data augmentation was performed. Specifically, as noted by the dataset documentation (<a href="https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/descr.html">Dataset Documentation</a>), for all problem instances there is a clause number-threshold after which the problem is highly unlikely to be satisfiable. So, before this threshold, the problem is most likely to be satisfiable and, for that reason, it is possible to augment the dataset in two ways :
<ol>
    <li>For each instance provide a satisfiable smaller version of it (the instance below the threshold).</li>
    <li>For each unsatisfiable instance provide another unsatisfiable instance (the instance just above the threshold).</li>
</ol>

In [4]:
pos_weight = dataset_processing()

Start the data processing...

Satisfiable CNFs   : 7402
Unsatisfiable CNFs : 5154

Ratio of SAT   : 0.5895
Ratio of UNSAT : 0.4105

Training set size: 11275
Validation set size: 1410
Test set size: 1407

Processing completed.


It should be noted that the LSTM model needs about 300 times more data that its parameters and in our case we have 970561 parameters and 14092 input data. **Thus, due to the amount of data, even after the augmentation, the model is not expected to reach high values in the evaluation metrics**. 

**Tune parameters**: Tune the parameters of the model (LSTM itself) as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specidically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Hidden Units</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [5]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight)

# Show best parameters
print(f'\nBest hyperparameters were: {best_parameters}\n')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.5723
Validation Loss : 0.5825

EPOCH | 1
Training Loss   : 0.5684
Validation Loss : 0.5785

EPOCH | 2
Training Loss   : 0.5663
Validation Loss : 0.5682

EPOCH | 3
Training Loss   : 0.5634
Validation Loss : 0.5806

EPOCH | 4
Training Loss   : 0.5635
Validation Loss : 0.5619

EPOCH | 5
Training Loss   : 0.5571
Validation Loss : 0.5903

EPOCH | 6
Training Loss   : 0.5533
Validation Loss : 0.5742

EPOCH | 7
Training Loss   : 0.5474
Validation Loss : 0.5842

EPOCH | 8
Training Loss   : 0.5397
Validation Loss : 0.5804

EPOCH | 9
Training Loss   : 0.5271
Validation Loss : 0.5955

EPOCH | 10
Training Loss   : 0.5163
Validation Loss : 0.5991

EPOCH | 11
Training Loss   : 0.5024
Validation Loss : 0.5958

EPOCH | 12
Training Loss   : 0.4818
Validation Loss : 0.6106

EPOCH | 13
Training

Training Loss   : 0.5670
Validation Loss : 0.5691

EPOCH | 4
Training Loss   : 0.5631
Validation Loss : 0.5799

EPOCH | 5
Training Loss   : 0.5594
Validation Loss : 0.5731

EPOCH | 6
Training Loss   : 0.5530
Validation Loss : 0.5786

EPOCH | 7
Training Loss   : 0.5478
Validation Loss : 0.5874

EPOCH | 8
Training Loss   : 0.5404
Validation Loss : 0.5943

EPOCH | 9
Training Loss   : 0.5317
Validation Loss : 0.5811

EPOCH | 10
Training Loss   : 0.5201
Validation Loss : 0.5880

EPOCH | 11
Training Loss   : 0.4975
Validation Loss : 0.5990

EPOCH | 12
Training Loss   : 0.4749
Validation Loss : 0.5993

EPOCH | 13
Training Loss   : 0.4457
Validation Loss : 0.6207

EPOCH | 14
Early stopping activated, with training and validation loss difference: 0.0021

Test number 10 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 105193

EPOCH | 0
Training Loss   : 0.5735
Validation Loss : 0.5870

Training Loss   : 0.5703
Validation Loss : 0.5792

EPOCH | 2
Training Loss   : 0.5690
Validation Loss : 0.5815

EPOCH | 3
Training Loss   : 0.5659
Validation Loss : 0.5816

EPOCH | 4
Training Loss   : 0.5637
Validation Loss : 0.5765

EPOCH | 5
Training Loss   : 0.5621
Validation Loss : 0.5721

EPOCH | 6
Training Loss   : 0.5581
Validation Loss : 0.5900

EPOCH | 7
Training Loss   : 0.5509
Validation Loss : 0.5925

EPOCH | 8
Training Loss   : 0.5435
Validation Loss : 0.5875

EPOCH | 9
Training Loss   : 0.5351
Validation Loss : 0.5693

EPOCH | 10
Training Loss   : 0.5232
Validation Loss : 0.6020

EPOCH | 11
Training Loss   : 0.5100
Validation Loss : 0.5939

EPOCH | 12
Early stopping activated, with training and validation loss difference: 0.0090

Test number 18 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 210641

EPOCH | 0
Training Loss   : 0.5719
Validation Loss : 0.5828



Training Loss   : 0.5570
Validation Loss : 0.5777

EPOCH | 8
Training Loss   : 0.5507
Validation Loss : 0.5949

EPOCH | 9
Training Loss   : 0.5426
Validation Loss : 0.5981

EPOCH | 10
Training Loss   : 0.5303
Validation Loss : 0.5745

EPOCH | 11
Training Loss   : 0.5136
Validation Loss : 0.5928

EPOCH | 12
Training Loss   : 0.4910
Validation Loss : 0.6132

EPOCH | 13
Early stopping activated, with training and validation loss difference: 0.0049

Test number 26 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 414625

EPOCH | 0
Training Loss   : 0.5770
Validation Loss : 0.5720

EPOCH | 1
Training Loss   : 0.5748
Validation Loss : 0.5865

EPOCH | 2
Training Loss   : 0.5732
Validation Loss : 0.5781

EPOCH | 3
Training Loss   : 0.5703
Validation Loss : 0.5910

EPOCH | 4
Training Loss   : 0.5686
Validation Loss : 0.6000

EPOCH | 5
Training Loss   : 0.5658
Validation Loss : 0.6002


Training Loss   : 0.5448
Validation Loss : 0.5913

EPOCH | 10
Training Loss   : 0.5332
Validation Loss : 0.5987

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0051

Test number 34 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 448417

EPOCH | 0
Training Loss   : 0.5772
Validation Loss : 0.5823

EPOCH | 1
Training Loss   : 0.5745
Validation Loss : 0.6019

EPOCH | 2
Training Loss   : 0.5732
Validation Loss : 0.5869

EPOCH | 3
Training Loss   : 0.5710
Validation Loss : 0.5789

EPOCH | 4
Training Loss   : 0.5686
Validation Loss : 0.5784

EPOCH | 5
Training Loss   : 0.5650
Validation Loss : 0.5798

EPOCH | 6
Training Loss   : 0.5619
Validation Loss : 0.5944

EPOCH | 7
Training Loss   : 0.5573
Validation Loss : 0.5775

EPOCH | 8
Training Loss   : 0.5519
Validation Loss : 0.5990

EPOCH | 9
Training Loss   : 0.5448
Validation Loss : 0.5913

E

...

Training Loss   : 0.0232
Validation Loss : 1.1662

EPOCH | 7
Training Loss   : 0.0162
Validation Loss : 1.2386

EPOCH | 8
Training Loss   : 0.0116
Validation Loss : 1.3108

EPOCH | 9
Training Loss   : 0.0087
Validation Loss : 1.3826

EPOCH | 10
Training Loss   : 0.0065
Validation Loss : 1.4465

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0301

Test number 50 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.5503
Validation Loss : 0.5795

EPOCH | 1
Training Loss   : 0.3435
Validation Loss : 0.6588

EPOCH | 2
Training Loss   : 0.1544
Validation Loss : 0.7896

EPOCH | 3
Training Loss   : 0.0741
Validation Loss : 0.9249

EPOCH | 4
Training Loss   : 0.0405
Validation Loss : 1.0437

EPOCH | 5
Training Loss   : 0.0245
Validation Loss : 1.1437

EPOCH | 6
Training Loss   : 0.0162
Validation Loss : 1.2279

E

Training Loss   : 0.5573
Validation Loss : 0.5945

EPOCH | 2
Training Loss   : 0.5396
Validation Loss : 0.6225

EPOCH | 3
Training Loss   : 0.5242
Validation Loss : 0.6109

EPOCH | 4
Training Loss   : 0.5017
Validation Loss : 0.6607

EPOCH | 5
Training Loss   : 0.4734
Validation Loss : 0.6951

EPOCH | 6
Training Loss   : 0.4352
Validation Loss : 0.7290

EPOCH | 7
Training Loss   : 0.4146
Validation Loss : 0.7298

EPOCH | 8
Training Loss   : 0.3604
Validation Loss : 0.8606

EPOCH | 9
Training Loss   : 0.3336
Validation Loss : 0.8569

EPOCH | 10
Training Loss   : 0.2993
Validation Loss : 0.9437

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0188

Test number 59 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.5793
Validation Loss : 0.5782

EPOCH | 1
Training Loss   : 0.5751
Validation Loss : 0.5766

E

Training Loss   : 0.0263
Validation Loss : 1.2842

EPOCH | 10
Training Loss   : 0.0688
Validation Loss : 1.0694

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.1056

Test number 67 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.5388
Validation Loss : 0.6476

EPOCH | 1
Training Loss   : 0.4411
Validation Loss : 0.7168

EPOCH | 2
Training Loss   : 0.3563
Validation Loss : 0.8367

EPOCH | 3
Training Loss   : 0.2992
Validation Loss : 1.0374

EPOCH | 4
Training Loss   : 0.2457
Validation Loss : 1.0795

EPOCH | 5
Training Loss   : 0.1949
Validation Loss : 1.3339

EPOCH | 6
Training Loss   : 0.1785
Validation Loss : 1.2905

EPOCH | 7
Training Loss   : 0.1429
Validation Loss : 1.3789

EPOCH | 8
Training Loss   : 0.1128
Validation Loss : 1.5032

EPOCH | 9
Training Loss   : 0.0877
Validation Loss : 1.6276

E

Training Loss   : 0.2963
Validation Loss : 0.6411

EPOCH | 4
Training Loss   : 0.1941
Validation Loss : 0.6979

EPOCH | 5
Training Loss   : 0.1355
Validation Loss : 0.7489

EPOCH | 6
Training Loss   : 0.1012
Validation Loss : 0.7919

EPOCH | 7
Training Loss   : 0.0799
Validation Loss : 0.8317

EPOCH | 8
Training Loss   : 0.0659
Validation Loss : 0.8674

EPOCH | 9
Training Loss   : 0.0563
Validation Loss : 0.9005

EPOCH | 10
Training Loss   : 0.0494
Validation Loss : 0.9280

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0016

Test number 76 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.4610
Validation Loss : 0.7140

EPOCH | 1
Training Loss   : 0.1389
Validation Loss : 1.1154

EPOCH | 2
Training Loss   : 0.0425
Validation Loss : 1.2469

EPOCH | 3
Training Loss   : 0.0245
Validation Loss : 1.4647

E

Dataset loading completed

Model loading...
Model loading completed

Number of model parameters: 102889

EPOCH | 0
Training Loss   : 0.5682
Validation Loss : 0.5749

EPOCH | 1
Training Loss   : 0.5637
Validation Loss : 0.5852

EPOCH | 2
Training Loss   : 0.5621
Validation Loss : 0.5908

EPOCH | 3
Training Loss   : 0.5631
Validation Loss : 0.5771

EPOCH | 4
Training Loss   : 0.5585
Validation Loss : 0.5810

EPOCH | 5
Training Loss   : 0.5538
Validation Loss : 0.5865

EPOCH | 6
Training Loss   : 0.5497
Validation Loss : 0.5778

EPOCH | 7
Training Loss   : 0.5439
Validation Loss : 0.5721

EPOCH | 8
Training Loss   : 0.5374
Validation Loss : 0.5906

EPOCH | 9
Training Loss   : 0.5342
Validation Loss : 0.5673

EPOCH | 10
Training Loss   : 0.5230
Validation Loss : 0.5908

EPOCH | 11
Early stopping activated, with training and validation loss difference: 0.0067

Best hyperparameters were: {'batch_size': 0, 'learning_rate': 0, 'weight_decay': 0, 'pos_weight': 0, 'model_hidden_units': 8, 'model

**Train with the best parameters**: The LSTM is trained using the selected parameters.

In [6]:
# Access the best parameters in order to train final model
with open('best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, make_err_logs=True)


Now training with the best parameters

Dataset loading...


ValueError: batch_size should be a positive integer value, but got batch_size=0

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [ ]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [ ]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded)

The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [ ]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))

Since, as one can notice, ***the results are not very promising using this model and this architecture***, with the current amount of data, the model is not tested on a Test Set from a different data distribution.